In [1]:
# Importin libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import models, transforms
import cv2
import os
import glob
import pandas as pd
from tqdm.notebook import tqdm 
from IPython.display import display

In [2]:
#pre-process    
def preprocess(img1,img2):
    H,W=416,416
    transform = transforms.ToTensor()

    img1=cv2.imread(img1)
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img1=cv2.resize(img1,(H,W))
    img1=transform(img1)
    img1=torch.unsqueeze(img1, 0)

    img2=cv2.imread(img2)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    img2=cv2.resize(img2,(H,W))
    img2=transform(img2)
    img2=torch.unsqueeze(img2, 0)
    
    
    return img1,img2

In [3]:
# CosineSimilarity using pytorch resnet pre-trained model
class FeaturesResNet(torch.nn.Module):
    def __init__(self):
        super(ReductionResNet, self).__init__()
        device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        resnet18 = models.resnet18(pretrained=True)
        num_ftrs = resnet18.fc.in_features
        self.resnet18 = resnet18.to(device)
        self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)
#         print(resnet18.eval())

    def forward(self,x):
        res="Not Match"
        x1,x2=x
        embedding1 = self.resnet18(x1)
        embedding2 = self.resnet18(x2)
        output = self.cos(embedding1, embedding2)
        if float(output[0]*100)>0.75:
            res="Match"
        return float(output[0]*100),res
                

# Inference

In [4]:
#reading images using its path
base=os. getcwd() 
Images_path=base+"\\Challenge_images\\*.jpg"
col_path=str(Images_path.split("\\")[:-1]).replace('[','').replace(']','').replace("'","").replace(",","\\").replace(" ","")

details_dic={}
model=FeaturesResNet()

In [5]:
model.eval()

ReductionResNet(
  (resnet18): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

In [6]:
for img1 in tqdm(glob.glob(Images_path)):
    name=img1.split("\\")[-1]
    details_dic[name]=[]
    for img2 in glob.glob(Images_path):
        compare=preprocess(img1,img2)
        value,res=model.forward(compare)
        details_dic[name].append((value,img2.split("\\")[-1]))

In [7]:
# visualize data using dataframe
columns=os.listdir(col_path)
columns=['Name']+columns
results = pd.DataFrame(columns = columns)

# appending rows
for i,j in details_dic.items():
    row_dic={"Name":i}
    for val,compare_img in j:
        row_dic[compare_img]=val
    results = results.append(row_dic,ignore_index = True)

display(results)    

,Name,0.jpg,1.jpg
0,0.jpg,100.000000,71.914001
1,1.jpg,71.914001,100.000000


In [8]:
results.to_csv('Cosine_similarity.csv')

# Results

In [9]:
total=0
for i in results.iterrows():
    count_=0
    for j in i[1]:
        if type(j) is str and j.endswith(".jpg"):pass
        elif j > 75:count_+=1
        else:pass   
    total+=count_ 
print("Total No. of Images: '",(results.shape[0])**2,"Images'.")    
print("Total no. of images where cosine val is greater than 0.75:'",total,"Images'.")    
print("'",round((total/(results.shape[0])**2)*100,2),"%' of images are similar to each other!")


Total No. of Images: ' 4 Images'.
Total no. of images where cosine val is greater than 0.75:' 2 Images'.
' 50.0 %' of images are similar to each other!


# Onxx conversion

In [1]:
# # Input to the model
# c1=torch.randn(1, 3, 416, 416,device="cpu")
# c2=torch.randn(1, 3, 416, 416,device="cpu")
# x = (c1,c2)

# # Export the model
# torch.onnx.export(model,x,"FeaturesResNet.onnx",verbose=True,input_names = ['input']) 

# Save Model

In [ ]:
torch.save(model.state_dict(), "./FeaturesResNet.pt")